In [1]:
# First dealing with all of the imports
import os
import cv2
import numpy as np
from tensorflow import keras as keras
from keras.utils import img_to_array
from keras.utils import to_categorical
from keras.applications.mobilenet_v2 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


D0819 01:56:12.766574724      15 config.cc:119]                        gRPC EXPERIMENT tcp_frame_size_tuning               OFF (default:OFF)
D0819 01:56:12.766610533      15 config.cc:119]                        gRPC EXPERIMENT tcp_rcv_lowat                       OFF (default:OFF)
D0819 01:56:12.766613976      15 config.cc:119]                        gRPC EXPERIMENT peer_state_based_framing            OFF (default:OFF)
D0819 01:56:12.766616746      15 config.cc:119]                        gRPC EXPERIMENT flow_control_fixes                  ON  (default:ON)
D0819 01:56:12.766619051      15 config.cc:119]                        gRPC EXPERIMENT memory_pressure_controller          OFF (default:OFF)
D0819 01:56:12.766621721      15 config.cc:119]                        gRPC EXPERIMENT unconstrained_max_quota_buffer_size OFF (default:OFF)
D0819 01:56:12.766624287      15 config.cc:119]                        gRPC EXPERIMENT new_hpack_huffman_decoder           ON  (default:ON)
D0819 01:56:12.

In [2]:
# Loading and Preprocessing the Dataset 
import xml.etree.ElementTree as ElemTree
from collections import OrderedDict
# Images and labels list
images = []
labels = []
data = OrderedDict()

annotation_path = '/kaggle/input/face-mask-detection/annotations'
images_path = '/kaggle/input/face-mask-detection/images'

# Function to parse the xml file 
def parse_xml(xml_file):
    tree = ElemTree.parse(xml_file)
    root = tree.getroot()

    bounding_boxes = []
    temp_labels = []

    for object in root.findall('object'):
        temp_labels.append(object[0].text)
        xmin = int(object[5][0].text)
        ymin = int(object[5][1].text)
        xmax = int(object[5][2].text)
        ymax = int(object[5][3].text)
        bounding_boxes.append([xmin, ymin, xmax, ymax])

    return bounding_boxes, temp_labels

# Looping through the annotations file directory 
for xml_file in os.listdir(annotation_path):
    if xml_file.endswith(".xml"):
        image_path = os.path.join(images_path, xml_file[:-4] + ".png")
        filename = xml_file[:-4] + ".png"
        xml_path = os.path.join(annotation_path, xml_file)

        image = cv2.imread(image_path)
        bounding_boxes, xml_labels = parse_xml(xml_path)
        tuple = (image, bounding_boxes, xml_labels)
            

    data[filename] = tuple

libpng warning: iCCP: Not recognizing known sRGB profile that has been edited


In [4]:
data_pre_list = []
labels_pre_list = []

for key in data.keys():
    image, boxes, image_labels = data[key]
    for box, label in zip(boxes, image_labels):
        data_pre_list.append((image, box))
        labels_pre_list.append(label)

train_data, val_data, train_labels, val_labels = train_test_split(data_pre_list, labels_pre_list, test_size=0.2, random_state=42)


In [ ]:
# Encoding the labels (with mask, no mask, wearing a mask incorrectly)
# lb = LabelBinarizer()
# labels = lb.fit_transform(labels).astype(int)

In [ ]:
# print(images.shape)
# print(labels.shape)
# print(lb.classes_)

In [ ]:
# Splitting the dataset into training and testing sets
# (train_images, test_images, train_labels, test_labels) = train_test_split(images, labels, test_size = 0.30, stratify=labels, random_state = 42)
# Test size of 0.20 means that 20% of data is for the test, and the rest is for the training set. 
# Picked a ranom state for reproduciablity if needed.

In [ ]:
# Data augmentation step.
# Not entirely needed but helps to increase the diversity of the dataset
# data_aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15, horizontal_flip=True, fill_mode="nearest")

In [ ]:
# train_data_gen = data_aug.flow(train_images, train_labels, batch_size=4)

In [8]:
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

input_size = (300, 300)

le = LabelEncoder()

# Fit the LabelEncoder with all the labels
le.fit(train_labels)
le.fit(val_labels)

train_images = []
train_label = []

for (image, box), label in zip(train_data, train_labels):

    x, y, width, height = box

    cropped_image = image[y:y+height, x: x+width]

    resized_image = cv2.resize(cropped_image, input_size)
    normalized_image = resized_image / 255.0

    # Transform the label using the fitted LabelEncoder
    encoded_label = le.transform([label])
    one_hot_label = to_categorical(encoded_label, num_classes=3)

    train_images.append(normalized_image)
    train_label.append(one_hot_label[0])

train_images = np.array(train_images)
train_label = np.array(train_label)

eval_images = []
eval_label = []

for (image, box), label in zip(val_data, val_labels):

    x, y, width, height = box

    cropped_image = image[y:y+height, x: x+width]

    resized_image = cv2.resize(cropped_image, input_size)
    normalized_image = resized_image / 255.0

    # Transform the label using the fitted LabelEncoder
    encoded_label = le.transform([label])
    one_hot_label = to_categorical(encoded_label, num_classes=3)

    eval_images.append(normalized_image)
    eval_label.append(one_hot_label[0])

eval_images = np.array(eval_images)
eval_label = np.array(eval_label)


In [9]:
print(val_labels)
for item in eval_label:
    print(item)

print(le.classes_)


['without_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'without_mask', 'with_mask', 'without_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'without_mask', 'with_mask', 'without_mask', 'with_mask', 'with_mask', 'with_mask', 'without_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'without_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'without_mask', 'with_mask', 'without_mask', 'with_mask', 'with_mask', 'mask_weared_incorrect', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'without_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'without_mask', 'with_mask', 'with_mask', 'without_mask', 'without_mask', 'with_mask', 'with_mask',

In [14]:
import tensorflow as tf
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
print('TPU Cluster Spec:', tpu.cluster_spec())


TPU Cluster Spec: ClusterSpec({})


In [18]:
from keras.applications import InceptionV3
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout

from keras.optimizers import Adam
from keras.losses import categorical_crossentropy

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)


with tpu_strategy.scope():

    base_model = InceptionV3(include_top=False, weights='imagenet', input_shape=(300, 300, 3))

    # Freeze base model layers
    for layer in base_model.layers:
        layer.trainable = False

    # Add custom layers on top
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x) 
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    predictions = Dense(3, activation='softmax')(x)

    # Create full model 
    model = Model(inputs=base_model.input, outputs=predictions)

    # Compile model
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss=categorical_crossentropy,
                  metrics=['accuracy'])


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local
INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:6, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


In [23]:
from keras.callbacks import LearningRateScheduler
import math
# Compile model

# Step decay schedule  
def step_decay(epoch):
  initial_lrate = 0.0001
  drop = 0.5
  epochs_drop = 10.0
  lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
  return lrate

# Learning rate scheduler callback  
lr_scheduler = LearningRateScheduler(step_decay)

model.fit(
  train_images,
  train_label, 
  epochs=50,
  batch_size=32,
  validation_data=(eval_images, eval_label),
  callbacks=[lr_scheduler]
)


Epoch 1/20
102/102 [==============================] - 7s 69ms/step - loss: 0.1872 - accuracy: 0.9306 - val_loss: 0.6743 - val_accuracy: 0.8123 - lr: 1.0000e-04
Epoch 2/20
102/102 [==============================] - 5s 46ms/step - loss: 0.1907 - accuracy: 0.9257 - val_loss: 0.6573 - val_accuracy: 0.8000 - lr: 1.0000e-04
Epoch 3/20
102/102 [==============================] - 5s 47ms/step - loss: 0.1762 - accuracy: 0.9343 - val_loss: 0.6722 - val_accuracy: 0.8037 - lr: 1.0000e-04
Epoch 4/20
102/102 [==============================] - 5s 46ms/step - loss: 0.1455 - accuracy: 0.9521 - val_loss: 0.7216 - val_accuracy: 0.8196 - lr: 1.0000e-04
Epoch 5/20
102/102 [==============================] - 5s 46ms/step - loss: 0.1348 - accuracy: 0.9493 - val_loss: 0.7919 - val_accuracy: 0.8160 - lr: 1.0000e-04
Epoch 6/20
102/102 [==============================] - 5s 47ms/step - loss: 0.1265 - accuracy: 0.9539 - val_loss: 0.8224 - val_accuracy: 0.8123 - lr: 1.0000e-04
Epoch 7/20
102/102 [====================

In [25]:
test_loss, test_acc = model.evaluate(train_images, train_label)
print('Test accuracy:', test_acc)

102/102 [==============================] - 4s 33ms/step - loss: 0.0070 - accuracy: 1.0000
Test accuracy: 1.0


In [26]:
model.save('modelImproved')

INFO:tensorflow:Assets written to: modelImproved/assets


INFO:tensorflow:Assets written to: modelImproved/assets
